In [9]:
import requests

PHI3_URL = "https://cu-vertical-dimensional-continuity.trycloudflare.com/phi3/generate"
PHI3_TOKEN = "AEulafWZQ1xhIkeLgjcp5nhL7d0cLPWiXAU34DTVvXI"  # Use the Token provided in the API docs: https://github.com/adventofmultimodalai/resources/blob/main/api.md#phi3-api-guide


def call_api(system, prompt, max_tokens, return_sentences_count=1, context=""):
    headers = {
        "Authorization": f"Bearer {PHI3_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {
        "inputs": f"<|system|>\n{system}<|end|>\n<|context|>{context}<|end|>\n<|user|>\n{prompt}<|end|>\n<|assistant|>",
        "parameters": {"max_new_tokens": max_tokens, "temperature": 0.1},  # Limit to a few words
    }
    response = requests.post(PHI3_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "").split(". ")[:return_sentences_count]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []


print(call_api("answer the user's query. Caution: use strictly upto 100 tokens", "what is a dog?", 100, 2))

['A dog is a domesticated mammal, scientifically known as Canis lupus familiaris', 'It belongs to the Canidae family, which also includes wolves, foxes, and other similar animals']


In [13]:
def generate_story_outline(prompt):
    headers = {
        "Authorization": f"Bearer {PHI3_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {
        "inputs": f"<|system|>\nYou are a creative manga story writer. Also breakdown the story into independent parts such that each can be represented by a picture. <|end|>\n<|user|>\n{prompt}<|end|>\n<|assistant|>",
        "parameters": {"max_new_tokens": 300},
    }
    response = requests.post(PHI3_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "").split("\n")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []


class Manga:
    def __init__(self, story=None):
        self.story = story
        self.strip_shared_idx = 0
        self.strips = []
        self.strip_image_descriptions = []
        self.title = ""
        self.poster = ""
        if story is None:
            return
        self.strips = []
        self.title = story[0]
        for para in story:
            if para[:8] == "Picture:":
                self.strips.append(para)
        self.poster = call_api(
            "You are a creative director who is assigned the task to create posters",
            f"Create a poster for the following story, described in 3 sentences: {story}",
            40,
            3,
        )

    def __str__(self):
        rt = ""
        rt += "Title" + self.title + "\n"
        rt += "Poster details" + self.poster + "\n"
        for strip in self.strips:
            rt += strip + "\n"
        return rt

    def generate_strip_descriptions(self):
        for strip in self.strips:
            print("strip:", strip)
            called = call_api(
                "You are a manga artist. You know the best way to represent a scene or a story part as an image. Your output will be used to generate images. Therefore, you need to be very descriptive and clear. Your sentences must be self-contained and should not require any additional context.",           
                f"Identify the following scene, represent in a single image-generation prompt strictly limited to 200 characters only: {strip}",
                40,
                3,
                context=self.story
            )
            print("generated: ", called)
            self.strip_image_descriptions.append(called)

    def generate_poster(self):
        self.poster = ''.join(call_api(
            "You are a creative artist who creates cinematic posters",
            f"Create a poster description for the following story (that can be used for image generation) and limit it strictly to 200 characters: {self.strips}",
            40,
            3,
        ))

    def add_strip(self, strip_text):
        self.strips.append(strip_text)
        return

    def share_strip_with_idx(self, idx):
        return self.strip[idx]

    def share_strip(self):
        strip = self.strip[self.strip_shared_idx]
        self.strip_shared_idx = self.strip_shared_idx + 1
        return strip


# Example usage
prompt = "A doggy detective discovers they have supernatural abilities while navigating a noir-inspired world of secrets, danger, and high-stakes mysteries in a rain-soaked city"
story_outline = [
    'Title: "Shadows & Bones: The Canine Chronicles"',
    "",
    "Part 1: The Awakening",
    "",
    "Picture: A dark, rain-soaked cityscape with neon signs flickering. A small, scruffy dog with a detective's hat and trench coat stands under an awning, observing the bustling streets.",
    "",
    'Story: Our protagonist, a doggy detective named "Bones," lives in a noir-inspired world where secrets and danger lurk around every corner. Bones has always been a bit different from other dogs, with a keen sense of intuition and an uncanny ability to sense danger. One night, while investigating a mysterious disappearance, Bones stumbles upon a hidden alley where a strange, glowing stone lies. As Bones touches the stone, they feel a surge of power coursing through their veins. From that moment on, Bones discovers they possess supernatural abilities, including telepathy, enhanced senses, and the power to communicate with other animals.',
    "",
    "Part 2: The Case of the Vanishing Vet",
    "",
    "Picture: A dimly lit veterinary clinic with a worried owner and a missing veterinarian. Bones, now aware of their newfound powers, senses a disturbance in the air.",
    "",
    "Story:",
]
if True:
    # regenerate only when needed to avoid long API
    story_outline = generate_story_outline(prompt)
print("Story Outline:", story_outline)
the_manga = Manga(story_outline)
for text in story_outline:
    if text[:8] == "Picture:":
        the_manga.add_strip(text[8:])
the_manga.generate_poster()
# print(the_manga)
print(the_manga.poster)
the_manga.generate_strip_descriptions()
print("\nStrip Desciptions: ", the_manga.strip_image_descriptions)

Story Outline: ['Title: "Shadows & Bones: The Canine Chronicles"', '', 'Part 1: The Awakening', '', "Picture: A dark, rain-soaked cityscape with neon signs flickering. A small, scruffy dog with a detective's hat and trench coat stands under an awning, observing the bustling streets.", '', 'Story: Our protagonist, a doggy detective named "Bones," lives in a noir-inspired world where secrets and danger lurk around every corner. Bones has always been a bit different from other dogs, with a keen sense of intuition and an uncanny ability to sense danger. One night, while investigating a mysterious disappearance, Bones stumbles upon a hidden alley where a strange, glowing stone lies. As Bones touches the stone, they feel a surge of power coursing through their veins. From that moment on, Bones discovers they possess supernatural abilities, including telepathy, enhanced senses, and the power to communicate with other animals.', '', 'Part 2: The Case of the Vanishing Vet', '', 'Picture: A diml

In [ ]:
FLUX_URL = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
FLUX_TOKEN = "calm-bold-tree-bd1b1c8bc141403231e7f0a8"  # Check the API docs for details.

def generate_image(description, output_file):
    headers = {
        "Authorization": f"Bearer {FLUX_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "prompt": description[:200],  # Max 200 characters for prompt
        "img_size": 512,
        "guidance_scale": 7.5,
        "num_inference_steps": 50
    }
    response = requests.post(FLUX_URL, headers=headers, json=payload)
    if response.status_code == 200:
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"Image saved to {output_file}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Example usage
generate_image(the_manga.poster, "poster.png")
for idx, strip in enumerate(the_manga.strip_image_descriptions):
    generate_image(str(strip), f"scene_{idx}.png")

In [18]:
print(the_manga.poster)

"Rain-drenched cityscape: Detective dog eyes neon chaosVet clinic: Bones sense the missing vet's shadowMystery unf
